In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
df = pd.read_csv('../input/shopee-code-league-20/_DA_Logistics/delivery_orders_march.csv')
pd.set_option('display.max_colwidth', None)
df['1st_deliver_attempt'] = df['1st_deliver_attempt'].astype(int)
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].dropna().astype(int)
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'], unit = 's')
df['pick'] = pd.to_datetime(df['pick'], unit = 's')
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'], unit = 's')
df['buyeraddress'] = df['buyeraddress'].str.split().str[-1].str.upper()
df['selleraddress'] = df['selleraddress'].str.split().str[-1].str.upper()
df['timediff'] = np.busday_count(df['pick'].values.astype('datetime64[D]'), df['1st_deliver_attempt'].values.astype('datetime64[D]'), weekmask='1111110')

def checkdays(buyer, seller):
    if buyer == 'MANILA':
        if seller == 'MANILA':
            days = 3
        if seller == 'LUZON':
            days = 5
        if seller == 'VISAYAS':
            days = 7
        if seller == 'MINDANAO':
            days = 7
        
    if buyer == 'LUZON':
        if seller == 'MANILA':
            days = 5
        if seller == 'LUZON':
            days = 5
        if seller == 'VISAYAS':
            days = 7
        if seller == 'MINDANAO':
            days = 7
            
    if buyer == 'VISAYAS':
        days = 7
            
    if buyer == 'MINDANAO':
        days = 7
    
    return days

df['daylimit'] = np.vectorize(checkdays)(df['buyeraddress'], df['selleraddress'])
tempdf = df
tempdf['2nd_deliver_attempt'] = tempdf['2nd_deliver_attempt'].fillna(0)
df['2nddiff'] = np.busday_count(tempdf['1st_deliver_attempt'].values.astype('datetime64[D]'), tempdf['2nd_deliver_attempt'].values.astype('datetime64[D]'), weekmask='1111110')

def islate(timediff, daylimit, seconddiff):
    if seconddiff > 3:
        return 1
    if timediff > daylimit:
        return 1
    else:
        return 0
df['islate'] = np.vectorize(islate)(df['timediff'], df['daylimit'], df['2nddiff'])
out = df[['orderid', 'islate']]
out